# 18460 Project

In [1]:
from client import *
import numpy as np
from problem import *
from server import *

First, we load the dataset and partition it

In [2]:
# TODO: load (and format) and partition dataset

Then, we define our optimization problem

In [3]:
# TODO: define cost functions and optimization hyperparameters

hyper_parameters = {
    # Symmetric PSD matrix
    "Q": np.array([
        [1, 2],
        [2, 1]
    ]),
    "penalty": 10
}

# simple quadratic cost
def cost(x, params):
    return x.T @ params["Q"] @ x

def cost_grad(x, params):
    return 2 * params["Q"] @ x

def cost_hessian(x, params):
    return 2 * params["Q"]

problem = OptimizationProblem(tol=1e-3, max_iter=300, loss=cost, loss_grad=cost_grad, 
                              loss_hessian=cost_hessian, hyper_parameters=hyper_parameters)

Solve iteratively using federated learning

In [4]:
# TODO: create server and initialize clients, before iteratively updating the consensus variables
server = Server(problem)
server.connect_clients() # add parameters and communicate datasets

k = 0
while k < problem.max_iter and server.delta > problem.tol:
    # perform new iteration
    server.run_iteration()
    
    k = k + 1
    
if server.delta > problem.tol:
    raise Exception("Did not converge")

RuntimeError: No clients initialized for the server, cannot update consensus